# Composite

## O que é?

O padrão _composite_ descreve uma forma de agrupar objetos em uma arquitetura de árvore de tal forma a tornar o comportamento de elementos individuais e grupos indistintos para o cliente.

Esse padrão também permite que elementos sejam agrupadas de forma recursiva, permitindo a criação de grupos mais complexos.

## Por quê?

Muitas vezes classes diferentes containers e primitivos apresentam comportamentos muito similares, porém ao tratá-los de forma diferente, a complexidade do código aumenta.

## Estrutura

![figura](./assets/composite_struct.png)

## Exemplo

Nós queremos criar um jogo de RPG. Onde nos temos aventureiros que se organizam numa party.

In [30]:
import abc

class Character:
    def __init__(self, name: str, strength: int, magic: int):
        self._name = name
        self._strength = strength
        self._magic = magic
        
    def __hash__(self):
        return hash(self.name)
        
    @property
    def name(self) -> str:
        return self._name
        
    @abc.abstractmethod
    def attack(self) -> float:
        """
        Character attacks.
        """
        pass
    
    
class Warrior(Character):
    def attack(self):
        return self._strength * 2
    

class Wizzard(Character):
    def attack(self):
        return self._magic * 2.5
    
    
class Scout(Character):
    def attack(self):
        return self._strength * .7
        

class Party:
    def __init__(self, name: str, *members: 'Characters'):
        self._name = name
        self._members = set(members)
        
    def __hash__(self):
        return hash(self.name)

    @property
    def name(self) -> str:
        return f"Party {self._name}"
        
    
    def team_attack(self) -> float:
        return sum(member.attack() for member in self._members)
        
    def recruit(self, *adventurers: 'Character') -> 'Adventurers':
        """
        Recruit adventurers to your group. Returns 
        """
        return Party(self.name, self._members + set(adventurers))

    def disband(self, *adventurers: 'Character') -> 'Adventurers':
        """
        Adventurers go their own way.
        """
        return Party(self.name, self._members - set(adventurers))
    

class Guild:
    def __init__(self, name: str, *parties: 'Party'):
        self.name = name
        self._parties = set(parties)
        
    def __hash__(self):
        return hash(self.name)

    @property
    def name(self) -> str:
        return f"Guild {self._name}"
        
    def guild_attack(self) -> float:
        return sum(party.team_attack() for party in self._parties)
        
    def form_alliance(self, *parties: 'Party') -> 'Party':
        """
        Recruit adventurers to your group. Returns 
        """
        return Guild(self.name, self._parties + set(parties))

    def break_alliance(self, *parties: 'Party') -> 'Party':
        """
        Adventurers go their own way.
        """
        return Guild(self.name, self._parties - set(parties))

Vamos atacar um monstro.

In [31]:
def fight(char: Character, enemy: Character):
    print(f"{char.name} caused {char.attack()} pts of damage.")
    print(f"{enemy.name} caused {enemy.attack()} pts of damage.")

conan = Warrior("Conan", 20, 3)
bandit = Scout("Bandit", 10, 5)

fight(conan, bandit)

Conan caused 40 pts of name.
Bandit caused 7.0 pts of name.


Após formar uma aliança com merlin, eles voltam à luta.

In [32]:
merlin = Wizzard("Merlin", 3, 30)

party = Party("Sword and magic", conan, merlin)
fight(party, bandit)

AttributeError: 'Party' object has no attribute 'attack'

Temos que adaptar a função `fight`

In [34]:
def fight(char: 'Any', enemy: Character):
    if isinstance(char, Character):
        dmg = char.attack()
    elif isinstance(char, Party):
        dmg = char.team_attack()
    else:
        dmg = char.guild_attack()
    
    print(f"{char.name} caused {dmg} pts of damage.")
    print(f"{enemy.name} caused {enemy.attack()} pts of damage.")
    
fight(party, bandit)

Party Sword and magic caused 115.0 pts of name.
Bandit caused 7.0 pts of name.


Usando o padrão composite.

In [46]:
class Adventurers(metaclass=abc.ABCMeta):
    def __init__(self, name: str):
        self._name = name
        
    def __hash__(self):
        return hash(self.name)
    
    @property
    def name(self) -> str:
        return self._name
    
    @abc.abstractmethod
    def attack(self) -> float:
        """
        The character performs an attack.
        """
        pass
    
    def recruit(self, *adventurers: 'Adventurers') -> 'Adventurers':
        """
        Recruit adventurers to your group. Returns 
        """
        return Party(self.name, self.members + set(adventurers))

    def disband(self, *adventurers: 'Adventurers') -> 'Adventurers':
        """
        Adventurers go their own way.
        """
        return Party(self.name, self.members - set(adventurers))


class Group(Adventurers):
    prefix = None
    
    def __init__(self, name, *members):
        super().__init__(name)
        self.members = set(members)
        
    @property
    def name(self):
        return f'{self.prefix} "{self._name}"'
    
    def attack(self):
        """
        The character performs an attack.
        """
        base_damage = sum(member.attack() for member in self.members)
        combo_multi = 1 + 0.05 * len(self.members)
        return base_damage * combo_multi
    
class Party(Group):
    prefix = "Party"
    
    
class Guild(Group):
    prefix = "Guild"
        
    
class Character(Adventurers):
    
    def __init__(self, name, strength, magic):
        super().__init__(name)
        self._strength = strength
        self._magic = magic
    
    def recruit(self, *adventurers):
        return Party(f"{self.name}'s party", self, *adventurers)
    
    def disband(self, *adventurers):
        return AttributeError("No one to disband.")
        
class Warrior(Character):
    def attack(self):
        return self._strength * 2
    

class Wizzard(Character):
    def attack(self):
        return self._magic * 2.5
    
    
class Scout(Character):
    def attack(self):
        return self._strength * .7

Vamos começar nossa aventura.

In [43]:
def fight(char: Character, enemy: Character):
    print(f"{char.name} caused {char.attack()} pts of damage.")
    print(f"{enemy.name} caused {enemy.attack()} pts of damage.")

In [44]:
conan = Warrior("Conan", 20, 4)
bandit = Scout("Bandit", 10, 5)

fight(conan, bandit)

Conan caused 40 pts of damage.
Bandit caused 7.0 pts of damage.


In [50]:
merlin = Wizzard("Merlin", 5, 25)

party = conan.recruit(merlin)
fight(party, bandit)

Party "Conan's party" caused 112.75000000000001 pts of damage.
Bandit caused 7.0 pts of damage.


In [52]:
musashi = Scout("Musashi", 20, 10)
suzano = Scout("Suzano", 18, 12)
wanderers = Party("Wanderers", musashi, suzano)

guild = Guild("Alliance", party, wanderers)

fight(guild, bandit)

Guild "Alliance" caused 156.21100000000004 pts of damage.
Bandit caused 7.0 pts of damage.


## Pros e cons

__Pros__
- Ter coleções e unidades intercambiáveis, muitas vez torna o código mais simples e evita repetição (e.g. numpy arrays).
- Os componentes podem ser compostos de forma indefinida, criando estruturas mais complexas (e.g. loggers).
- O fato dos componentes implementarem uma interface comum torna fácil a adição de novas classes.

__Cons__
- Por vezes é necessário que uma hierarquia seja estabelecida, porém esse padrão não preve tal conceito (e.g. containers dentro de containers).
- A interface pode ser tornar bastante grande e certos comportamentos específicos a uma classe são difíceis de serem implementados (e.g. methods _add_ e _remove_).

## Possíveis _use cases_

- Estruturas que precisão de valores agregados em camadas (e.g. lucro da companhia vs região vs de uma loja).
- Casos em que é necessário propagar ação para apenas um grupo de objetos.

## Detalhes de implementação
- Muitas vezes é útil manter uma referência ao objeto pai dentro do objeto filho (e.g. invalidar o cache).
- Compartilhar componentes reduz o uso de memória, porém torna mais difícil a propagação de mensagens e pode causar efeitos secundários (e.g. remover uma _leaf_ compartilhada).
- Nem todas a operações são relevantes para todos os componentes. Todos esses métodos devem ser implentados (e.g. _add_ e _remove_).
- É preciso pensar em _caching_ e boas estruturas de dados para os objetos filhos para garantir boa performance.

## Perguntas
- Como tratar elementos que precisem de características especiais?
- Quais são as alternativas a esse padrão?